# Projeto 2 - Ciência dos Dados

Nome: Amanda Rosa do Carmo

Nome: Beatriz Cabral

Nome: Sophia Kerber Shigueoka

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [3]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [4]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [55]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

___
## Autenticando no  Twitter

* Conta: ***@KerberSophie***

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @KerberSophie

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [6]:
#Produto escolhido:
produto = '13 Reasons Why'

#Quantidade mínima de mensagens capturadas:
#Escolhemos 1480 a partir de um método empírico. Quando testamos com 1000 não conseguimos tweets suficientes, e chegamos
#a conclusão de que o problema era proveniente do set(), que diminuía a quantidade de tweets baixados. Assim, decidiu-se
#aumentar a quantidade de tweets para 1500, já que com 1000 ficaram falatando 360 tweets. Com 1500, ficamos com 20 tweets a mais.
#como eram somente 20 tweets, imaginamos que poucos deles seriam retweets, então tiramos exatamente 20 tweets dos 1500.
#Com 1480 tweets, conseguimos 399 tweets de teste, ou seja, faltou somente 1. não achamos que 1 tweets afetará tanto a base
# de dados, então decidimos manter os 1480.
n = 1480
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
list = []
shuffle(msgs)

#Usando o set() para tirar os retweets
listaset = set(msgs)

#Adicionando o resultado do set() a uma lista para evitar o erro TypeError: 'set' object is not subscriptable
for value in listaset:
    list.append(value)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/search/tweets.json?q=13+Reasons+Why&lang=pt (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

Salvando os dados em uma planilha Excel:

In [41]:
#Linhas abaixo foram comentadas uma vez que o conjunto de dados era o mesmo, mas o número de tweets foi alterado.

# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
writer = pd.ExcelWriter('{0} novo.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
dft = pd.DataFrame({'Treinamento' : pd.Series(list[:t])})
dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

dfc = pd.DataFrame({'Teste' : pd.Series(list[t:])})
dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [33]:
excel = pd.read_excel('13 Reasons Why novo.xlsx', sheet_name='Treinamento')

excel.head()

# excel.set_index('Categoria')




,Treinamento,Categoria
0,alguém aq ja assistiu a 3° temp de 13 reasons ...,0
1,(na 3° temporada de 13 reasons why) eu estou t...,R
2,13 ep de 13 reasons why p tentar fazer vc sent...,N
3,nossa nem sabia que tinha lançado a 3° tempora...,0
4,estava começando a ver o primeiro episódio de...,N


In [64]:
Neutro = " ".join(excel[excel.Categoria==0].Treinamento)

Reacao = " ".join(excel[excel.Categoria=='R'].Treinamento)

Negativo = " ".join(excel[excel.Categoria=='N'].Treinamento)

Positivo = " ".join(excel[excel.Categoria=='P'].Treinamento)

# Neutro = re.sub(r'^https?:\/\/.*[\r\n]*', '', Neutro, flags=re.MULTILINE)
# Reacao = re.sub(r'^https?:\/\/.*[\r\n]*', '', Neutro, flags=re.MULTILINE)
# Negativo = re.sub(r'^https?:\/\/.*[\r\n]*', '', Neutro, flags=re.MULTILINE)
# Positivo = re.sub(r'^https?:\/\/.*[\r\n]*', '', Neutro, flags=re.MULTILINE)

semhttp = re.sub(r'http\S+', '', Neutro)

Neutro_limpo = cleanup(Neutro)
Reacao_limpo = cleanup(Reacao)
Negativo_limpo = cleanup(Negativo)
Positivo_limpo = cleanup(Positivo)

Neutro_rel = Neutro_limpo.split()
Reacao_rel = Reacao_limpo.split()
Negativo_rel = Negativo_limpo.split()
Positivo_rel = Positivo_limpo.split()

freq_rel = pd.Series(Neutro_rel).value_counts(True)
freq_rel

reasons          0.047192
13               0.046802
de               0.042122
why              0.039002
a                0.027301
e                0.023401
//t              0.022231
https            0.022231
temporada        0.014821
que              0.012871
eu               0.010530
o                0.010530
assistir         0.009750
elite            0.008970
sobre            0.007800
não              0.007800
é                0.007800
3                0.006240
•                0.006240
rt               0.006240
vou              0.006240
pra              0.006240
se               0.005850
do               0.005850
2                0.005850
why,             0.005460
ver              0.005460
elite,           0.005070
mas              0.005070
to               0.005070
                   ...   
is               0.000390
indico           0.000390
×                0.000390
himym,…          0.000390
ducha            0.000390
’13              0.000390
algebra          0.000390
bacurau     

In [35]:
#Lista de palavras para cada categoria
palavras_pos = []
palavras_neg = []
palavras_neut = []
palavras_reac = []


#Criando dicionários para as frequências absolutas de cada palavra
freq_pos = {}
freq_neg = {}
freq_neut = {}
freq_reac = {}

n = 0
#Para cada tweet:
for linha in excel['Treinamento']:
    print(linha)
#Se a palavra for positiva:
#     if excel.iloc[n] == 'P':
#         for palavra in tweet:
#             if palavra not in palavras_pos:
#                 palavras_pos.append(palavra)
#             elif palavra in palavras_pos:
#                 freq_pos[palavra] += 1
                
# #Se a palavra for negativa:
                
#     if excel.iloc[n] == 'N':
#         for palavra in tweet:
#             if palavra not in palavras_neg:
#                 palavras_neg.append(palavra)
#             elif palavra in palavras_neg:
#                 freq_neg[palavra] += 1
                
# #Se a palavra for neutra/irrelevante:
                
#     if excel.iloc[n] == '0':
#         for palavra in tweet:
#             if palavra not in palavras_neut:
#                 palavras_neut.append(palavra)
#             elif palavra in palavras_neut:
#                 freq_neut[palavra] += 1
                
# #Se a palavra for de reação:
                
#     if excel.iloc[n] == 'R':
#         for palavra in tweet:
#             if palavra not in palavras_reac:
#                 palavras_reac.append(palavra)
#             elif palavra in palavras_reac:
#                 freq_reac[palavra] += 1
                
# #Segue para o próximo tweet
# n += 1

alguém aq ja assistiu a 3° temp de 13 reasons why?? podem me falar antes q eu me encha de esperança e quebre a cara… https://t.co/yo0knm1ymi
(na 3° temporada de 13 reasons why) eu estou tipo "não sei como é que consigo ter tanta pena do bryce!!!"
13 ep de 13 reasons why p tentar fazer vc sentir pena de um estuprador, p colocar a culpa de um assassinato no cara… https://t.co/jvlc1jzxdf
nossa nem sabia que tinha lançado a 3° temporada de 13 reasons why
estava começando a ver  o primeiro episódio de  13 reasons why, mas  agora parei, tô cheia de medo kkk
to até agora vendo série, só vou ir dormir quando acabar de ver o último episódio de 13 reasons why
finalmente to vendo 13 reasons why e é bom.
rt @mlchelee: se vcs for assistir a 3 temp de 13 reasons why vão ficar meio perturbados né, mas vão parar pra pensar e ver que quase tudo é…
assistir 10 horas de video aula de algebra linear ou 10 horas de 13 reasons why
acabei a quarta temporada de vis a vis, acabei 13 reasons why e com esse temp

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**